## Import Dependencies

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import numpy as np
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt

## Run the webscraper

In [ ]:
# set up the driver
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome("/Users/macdkw/Downloads/chromedriver")

# Navigate to the page
def home():
    driver.get("https://www.texasmusicforms.com/csrrptUILpublic.asp")

# get results button function
def click_results_button():
    get_results_button = driver.find_element(by=By.CLASS_NAME, value="btn-success")
    get_results_button.click()


# "Search again" button function
def click_search_button():
    search_again_button = driver.find_element(by=By.NAME, value="chan")
    search_again_button.click()
    driver.implicitly_wait(10)

# find the drop down menus and fill them out
def form_completer(year, region, contest_item):
    year_menu = driver.find_element(by=By.NAME, value="yr")
    region_menu = driver.find_element(by=By.NAME, value="reg")
    event_menu = driver.find_element(by=By.NAME, value="ev")
    year_menu.send_keys(year)
    region_menu.send_keys(f"Region {region}")
    event_menu.send_keys("Band")

    # find the hidden drop down menu, and collect its contents
    contest_menu = driver.find_element(by=By.NAME, value="cn")
    contest_menu_contents = contest_menu.find_elements(by=By.TAG_NAME, value="option")
    contest_menu_length = len(contest_menu_contents)
    contest_menu.send_keys(contest_menu_contents[contest_item].text)


# scrape the data
def scraper_loop():
    # make sure that were seeing all results, not just the top 20
    show_all_button = driver.find_element(by=By.NAME, value="DataTables_Table_0_length")
    show_all_button.send_keys("All")
    # get the table
    table = driver.find_elements(by=By.TAG_NAME, value="tbody")[1]
    table_rows = table.find_elements(by=By.TAG_NAME, value="tr")
    # loop through table contents
    for row in table_rows:
        cells = row.find_elements(by=By.TAG_NAME, value="td")
        column_content = []
        for cell in cells:
            cell_content_full = cell.text
            cell_content = cell_content_full.split("\n")
            for i in cell_content:
                column_content.append(i)
            #column_content.append(cell_content)
        row_content.append(column_content)

# convert to dataframe
def convert_to_dataframe():
    df = pd.DataFrame(row_content)
    if len(df.columns) == 21:
        df.columns = columns_21
    elif len(df.columns) == 22:
        df.columns = columns_22
    # export to csv
    df.to_csv(f"full_run/texas_music_forms_{year}_{region}_{contest_item}_band.csv")

columns_21 = ['Event', 'School', 'TEA', 'City', 'Directors', 'Conference', 'Classification', 'Year', 'ID', 
    'Stage Judge 1', 'Stage Judge 2', 'Stage Judge 3', 'Stage Final', 
    'SR Judge 1', 'SR Judge 2', 'SR Judge 3', 'SR Final', 'Award', 
    'Selection 1', 'Selection 2', 'Selection 3']

columns_22 = ['Event', 'School', 'TEA', 'City', 'Directors', 'Conference', 'Classification', 'Year', 'ID', 
    'Stage Judge 1', 'Stage Judge 2', 'Stage Judge 3', 'Stage Final', 
    'SR Judge 1', 'SR Judge 2', 'SR Judge 3', 'SR Final', 'Award', 
    'Selection 1', 'Selection 2', 'Selection 3', 'oops']

row_content = []

# Define year and region variables
year = 2005
region = 0

for y in range(2005, 2023):
    for r in range(0, 33):
        try:
            contest_item = 1
            region = region + 1
            for i in range(1, 15):
                try:
                    home()
                    form_completer(year, region, contest_item)
                    click_results_button()
                    scraper_loop()
                    convert_to_dataframe()
                    contest_item = contest_item + 1
                    row_content = []
                except:
                    print(f"{year}, {region}, {contest_item}")
                    break
        except:
            break
    year = year + 1
    region = 0

In [2]:
### Define column headers for csv export
columns_21 = ['Event', 'School', 'TEA', 'City', 'Directors', 'Conference', 'Classification', 'Year', 'ID', 
    'Stage Judge 1', 'Stage Judge 2', 'Stage Judge 3', 'Stage Final', 
    'SR Judge 1', 'SR Judge 2', 'SR Judge 3', 'SR Final', 'Award', 
    'Selection 1', 'Selection 2', 'Selection 3']
columns_22 = ['Event', 'School', 'TEA', 'City', 'Directors', 'Conference', 'Classification', 'Year', 'ID', 
    'Stage Judge 1', 'Stage Judge 2', 'Stage Judge 3', 'Stage Final', 
    'SR Judge 1', 'SR Judge 2', 'SR Judge 3', 'SR Final', 'Award', 
    'Selection 1', 'Selection 2', 'Selection 3', 'oops']

## Group the CSVs together, and fix rows with Accompanist error

In [4]:
# list of merged files returned
files = glob.glob("full_run/*.csv")

# joining files with concat and read_csv
df = pd.concat(map(pd.read_csv, files), ignore_index=True)
df = pd.DataFrame(df)

# drop unnamed column
df.drop(columns=['Unnamed: 0'], inplace=True)

# select rows with acc column 
oops_df = df[df['oops'].notnull()]

# delete the acc columns and fix column names
oops_fix_df = oops_df.drop(columns=['Conference'])
oops_fix_df.columns = columns_21

# drop acc rows from df
df = df.loc[df['oops'].isnull() == True]

# add fixed df to df
df = pd.concat([df, oops_fix_df], ignore_index=True)

# drop oops column
df.drop(columns=['oops'], inplace=True)



## Clean the data further

In [5]:
# drop duplicate rows
df_clean = df
df_clean.drop_duplicates(keep='first', inplace=True)
df_clean.sort_values(by=['Classification', 'Year'], inplace=True)
df_clean



,Event,School,TEA,City,Directors,Conference,Classification,Year,ID,Stage Judge 1,...,Stage Judge 3,Stage Final,SR Judge 1,SR Judge 2,SR Judge 3,SR Final,Award,Selection 1,Selection 2,Selection 3
10728,100-Concert Band,Woodville High,TEA:,Woodville,Lou Ellis,AA,,2009,46343,2,...,2,2,1,1,1,1,1,Under The Double Eagle - March (Wagner/Glover),A Childhood Hymn (Holsinger),Knights of Dunvegan (Meyer)
11455,100-Concert Band,New Deal High School,TEA:,New Deal,James Nance / Kay Nance,A,,2009,45190,2,...,1,2,1,1,1,1,1,La Fiesta Brava (John Edmondson),Cumberland Cross (Strommen),Quintology (Meyer)
12859,100-Concert Band,Progreso High School,TEA:,Progreso,James Yates,AAA,,2009,48134,3,...,2,2,2,2,2,2,,Portrait of a Clown (Ticheli),Prospect (La Plant),March of Freedom (Robert M. Geisler)
13480,100-Concert Band,Bangs HS,TEA:,Bangs,Tommy Perkins / Charla Bretzke,AA,,2009,43637,1,...,1,1,1,1,1,1,A,Monterey March (LaPlante),Two British Folk Songs (Anon. or Trad./Del B...,A Hymn for Band (Stuart)
15386,100-Concert Band,Alicia R. Chacon,TEA:,El Paso,"Jose Barriga / Salvador Melendez, Louie Nava",C,,2010,59515,1,...,1,1,2,1,1,1,C,Lexington March (Edmondson),Voodoo Dance (del Borgo),Medieval Legend (Story)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34426,100-Concert Band,Warren Junior High School,TEA:,Warren,Carl Hooper / Russell Hopkins,AAA,Varsity C,2016,116407,4,...,4,4,2,1,1,1,2,Swashbuckler's march (Mark Williams),Courtlandt County Festival (Owens/ ),Sea Song Trilogy (McGinty/ )
45355,100-Concert Band,McCamey MS,TEA:,McCamey,Shawna Torres / German Torres,C,Varsity C,2018,129652,2,...,2,2,1,2,1,1,2,Forward March (Paul Murtha),Majestica (Gazlay),Dance Celebration (Smith)
59639,100-Concert Band,Alpine MS,TEA: 022 901041,Alpine,Marina Azar,C,Varsity C,2022,172064,1,...,1,1,1,1,1,1,C,Independentia March (Matt Conaway),Dorian chant (Bill Calhoun),Conviction (Larry Clark)
59894,100-Concert Band,Lake Air Montessori Magnet,TEA: 161914,Waco,Todd Modgling / Stephanie Fernihough,C,Varsity C,2022,175313,2,...,1,2,1,2,1,1,2,Spirit of America March (Ken Harris),Two Classic Miniatures (71) (Haydn/Williams),Nottingham Castle (12) (Daehn)


In [6]:
# select rows where selection 3 is null
df_clean_selection_3_null = df_clean[df_clean['Selection 3'].isnull()]
df_clean_selection_3_null

,Event,School,TEA,City,Directors,Conference,Classification,Year,ID,Stage Judge 1,...,Stage Judge 3,Stage Final,SR Judge 1,SR Judge 2,SR Judge 3,SR Final,Award,Selection 1,Selection 2,Selection 3
700,100-Concert Band,0,TEA:,,cathy,Varsity,2005,11109,1,1,...,1,1,1,1,1,TRC,Escapada (David Moore),Whitewater Festival (Shaffer),Declaration Overture (Smith),NaN
669,100-Concert Band,0,TEA:,,Mandi McCasland,Varsity,2005,11197,,,...,,,,,,,Valdres (Hannsen/Curnow),Down Longford Way (Grainger/Osmon),Signature (Van der Roost),NaN
1978,100-Concert Band,0,TEA:,,lflisowski,Varsity,2005,12205,1,1,...,1,2,2,2,2,RM1,"War March of the Tartars (King, K. L.)",Shepherd's Hey (Grainger/Rogers/Mas),Divertimento for Symphonic Winds and Percussi...,NaN
1854,100-Concert Band,0,TEA:,,sbostwick,Non-Varsity,2005,12811,4,4,...,4,4,4,4,4,-,Blue Ribbon March (Feldstein-O'Reilly),Sea Song Trilogy (McGinty),Two Minute Symphony (Margolis),NaN
994,100-Concert Band,0,TEA:,,cjcain,Varsity,2005,13014,2,2,...,2,1,1,1,1,RM1,Newcastle March (Vinson),The Battle Pavane (Susato/Margolis),Distant Horizons (Sweeney),NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27352,100-Concert Band,Del Rio High School,TEA:,Del Rio,Daniel White,Sub Non-Varsity,2014,92973,1,1,...,1,1,2,1,1,C,Broken Bow (Strommen/ ),Steel (Standridge/ ),Bonds of Unity (King/Swearingen),NaN
30444,100-Concert Band,Alvarado MS,TEA:,Alvarado,Kelli Bahner / Joe Gunn,Non-Varsity,2014,92415,1,1,...,1,2,2,2,2,2,Ghost Ship (Story/ ),Courtlandt County Festival (Owens/ ),Forward March (Murtha),NaN
30443,100-Concert Band,Alvarado MS,TEA:,Alvarado,Kelli Bahner / Joe Gunn,Varsity,2014,92418,1,1,...,1,1,1,1,1,C,Fields of Clover (Standridge/ ),Accolade (Himes/ ),The Band in the Square on the Fourth of July ...,NaN
27506,100-Concert Band,Stinson Middle School,TEA:,San Antonio,Kevin Leman / Alex Melendez,Non-Varsity,2014,95724,1,1,...,1,1,1,1,1,D,Forward March (Paul Murtha),Fortis (Gazlay/ ),Dance Celebration (Smith/ ),NaN


In [7]:
df_clean_selection_3_null['Selection 3'] = df_clean_selection_3_null['Selection 2']
df_clean_selection_3_null['Selection 2'] = df_clean_selection_3_null['Selection 1']
df_clean_selection_3_null['Selection 1'] = df_clean_selection_3_null['Award']
df_clean_selection_3_null['Award'] = df_clean_selection_3_null['SR Final']
df_clean_selection_3_null['SR Final'] = df_clean_selection_3_null['SR Judge 3']
df_clean_selection_3_null['SR Judge 3'] = df_clean_selection_3_null['SR Judge 2']
df_clean_selection_3_null['SR Judge 2'] = df_clean_selection_3_null['SR Judge 1']
df_clean_selection_3_null['SR Judge 1'] = df_clean_selection_3_null['Stage Final']
df_clean_selection_3_null['Stage Final'] = df_clean_selection_3_null['Stage Judge 3']
df_clean_selection_3_null['Stage Judge 3'] = df_clean_selection_3_null['Stage Judge 2']
df_clean_selection_3_null['Stage Judge 2'] = df_clean_selection_3_null['Stage Judge 1']
df_clean_selection_3_null['Stage Judge 1'] = df_clean_selection_3_null['ID']
df_clean_selection_3_null['ID'] = df_clean_selection_3_null['Year']
df_clean_selection_3_null['Year'] = df_clean_selection_3_null['Classification']
df_clean_selection_3_null['Classification'] = None

C:\Users\macdk\AppData\Local\Temp\ipykernel_7672\3267504115.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean_selection_3_null['Selection 3'] = df_clean_selection_3_null['Selection 2']
C:\Users\macdk\AppData\Local\Temp\ipykernel_7672\3267504115.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean_selection_3_null['Selection 2'] = df_clean_selection_3_null['Selection 1']
C:\Users\macdk\AppData\Local\Temp\ipykernel_7672\3267504115.py:3: SettingWithCopyWarning: 
A value is trying to be set o

## Combine cleaned rows and export full output to CSV

In [8]:
# drop df_clean rows where selection 3 is null
df_clean = df_clean[df_clean['Selection 3'].isnull() == False]

# add df_clean_selection_3_null to df_clean
df_clean = pd.concat([df_clean, df_clean_selection_3_null], ignore_index=True)

# Make df_clean column integers
df_clean['Year'] = df_clean['Year'].astype(int)

# sort df_clean by year
df_clean.sort_values(by=['Year'], inplace=True)

# convert to csv
df_clean.to_csv("full_output.csv")


In [9]:
# Select all 'Selection 1' rows as a list
selection_1_list = df_clean['Selection 1'].tolist()
# Select all 'Selection 2' rows
selection_2_list = df_clean['Selection 2'].tolist()
# Select all 'Selection 3' rows
selection_3_list = df_clean['Selection 3'].tolist()

# combine selection 1, 2, and 3 into one list
selection_list = selection_1_list + selection_2_list + selection_3_list

# convert list to dataframe
selection_df = pd.DataFrame(selection_list)
selection_df.columns = ['Selection']

# count the number of times each selection appears in selection_list
selection_count = selection_df['Selection'].value_counts()
selection_count

 Dance Celebration  (Smith)                                         776
 Moscow, 1941  (Balmages)                                           687
 Colliding Visions  (Balmages/ )                                    681
 Atlantis  (McGinty)                                                639
 Pinnacle  (Grice)                                                  592
                                                                   ... 
 The Trombone King (Karl King, arr. John P. Painter)                  1
 Children's March "Over the Hills and Far Away" (Percy Grainger)      1
 Discovery March (John Edmundson)                                     1
 Mr. Bartow (Larry Clark)                                             1
 Fidelity March (K.L. King/Milford)                                   1
Name: Selection, Length: 31209, dtype: int64

# Get the full PML list to CSV

In [11]:
# import all of the pml csv files into a dataframe
files = glob.glob("pml/*.csv")
pml_df = pd.concat([pd.read_csv(f) for f in files], ignore_index=True)
# conver pml_df to csv
pml_df.to_csv("pml_output.csv")